In [1]:
import yahoo_fin as yf
from yahoo_fin.stock_info import get_data
import time
import datetime
import pandas as pd
from yahoo_fin import stock_info as si

import tensorflow as tf 
import numpy as np
from tensorflow.keras import layers

import concurrent.futures

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# getting data

things we did to clean data:

1. shamelessly copy:
https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b

2. we removed the tickers with $

3. we added exception handeling that tries similer strings to the input (note that this is not implemented on the set itself but when fetching, leading to multiple requsts and potentialy to errors)
4.  we just drop exceptions that are found when getting the data from online  (these are diffrent every day)

# how are we keeping consistency? 
since the avilble stocks may change depending on the source of the data some extra steps need to be done in order to
alow saves to work seemlesly 


basic plan:

1. clean the data as much as possible 

2. save the embedings weights with their ticker string

3. backup everything on git to alow us to fix it



In [2]:
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

symbols = set.union( sym1, sym2, sym3, sym4 )
del(sym1, sym2, sym3, sym4,df1, df2, df3, df4)

In [3]:
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if (len( symbol ) > 4 and symbol[-1] in my_list or "$" in symbol):
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
sav_set.discard("") 

ticker_list=list(sav_set)

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1584 unqualified stock symbols...
There are 10628 qualified stock symbols...


In [4]:
start='01/01/2020' 
end='09/14/2022'

In [5]:
import concurrent.futures 

def get_seiries(ticker,start,end,entry="adjclose"):
    try:
        raw = get_data(ticker.split(".")[0], start_date=start, end_date=end)[entry]
    except: 
        try:
            raw = get_data(ticker.replace(".","-"), start_date=start, end_date=end)[entry]
        except:
            raw = get_data(ticker[0:4], start_date=start, end_date=end)[entry]
    return raw/raw[0]

def get_df(l,start,end,entry="adjclose"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        x=[executor.submit(lambda x: get_seiries(x,start,end,entry=entry),name)for name in l]
        x=[a.result() for a in x]
    for i,c in enumerate(x):
        c.name=l[i]
    return pd.DataFrame(x).transpose()

def tester_func(name,start=start,end=end,entry="adjclose"):
    try:
        c= get_seiries(name,start,end,entry)
        c.name=name
        return (True,c)
    except:
        return (False,name)

In [6]:
mini=ticker_list
errors=[]
data=[]
c=0
x=list(range(len(mini)))
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(len(mini)):
        x[i]=executor.submit(tester_func,mini[i])
    
    print("thread setup complete")
    
    for i in range(len(mini)):
        if(i%100==0):
            print(i)
        r=x[i].result()
        if (r[0]): 
            data.append(r[1])
        else:
            errors.append(r[1])
            c+=1
            print(f"error number {c}: name={errors[-1]}")

print("done")

thread setup complete
0
100
200
300
400
error number 1: name=CTEST.O
error number 2: name=EMP
500
600
700
800
error number 3: name=CTEST.V
error number 4: name=GRP.U
900
error number 5: name=ZEXIT
1000
1100
1200
error number 6: name=DEFI
1300
1400
1500
1600
error number 7: name=ZIEXT
1700
error number 8: name=CRBG
1800
error number 9: name=XTWY
1900
2000
error number 10: name=EAI
error number 11: name=ZXYZ.A
2100
2200
2300
2400
2500
error number 12: name=ZAZZT
2600
2700
2800
2900
error number 13: name=CTEST.L
3000
error number 14: name=ZBZX
3100
3200
3300
3400
3500
3600
error number 15: name=KDIV
3700
3800
error number 16: name=XONE
3900
error number 17: name=CBO
4000
4100
4200
4300
4400
error number 18: name=ZCZZT
4500
error number 19: name=ZVZZT
error number 20: name=THRD
4600
4700
4800
4900
5000
error number 21: name=LNKB
5100
error number 22: name=XTRE
5200
error number 23: name=ZXIET
5300
error number 24: name=ZXZZT
5400
error number 25: name=ATEST.C
error number 26: name=ZJZZT
er

In [7]:
data_df=pd.DataFrame(data).transpose()
data_df

,BLTSU,NHTC,PCSB,SLAB,EBIZ,IEV,OGN,PERI,MGYR,SIEB,...,GETY,GEL,EMLDU,KRNL,XEL,NXDT,XDSQ,IPOF.U,OMAB,VIDI
2020-01-02,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,...,NaN,1.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,1.000000,1.000000
2020-01-03,NaN,1.037037,0.995547,0.980407,0.995498,0.989224,NaN,1.009231,1.000000,0.980616,...,NaN,0.999527,NaN,NaN,1.004809,1.006780,NaN,NaN,0.984192,0.988641
2020-01-06,NaN,1.031482,0.992578,0.977198,0.994703,0.992816,NaN,1.123077,1.000000,1.003421,...,NaN,1.004255,NaN,NaN,1.003366,1.012994,NaN,NaN,0.978758,0.984332
2020-01-07,NaN,1.038889,0.974270,0.989697,1.004767,0.987112,NaN,1.193846,1.000000,0.978335,...,NaN,1.034516,NaN,NaN,1.001282,1.012429,NaN,NaN,0.970196,0.983549
2020-01-08,NaN,1.029630,0.980703,0.990710,1.008475,0.989013,NaN,1.273846,1.000000,0.969213,...,NaN,1.037825,NaN,NaN,1.000321,1.012994,NaN,NaN,0.985674,0.988641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,0.940114,1.211159,0.950144,1.003125,0.939027,0.941034,0.908283,3.290769,1.302233,0.183580,...,1.263366,0.665742,0.992177,1.026943,1.284394,0.881921,0.950468,0.973710,0.880454,0.886147
2022-09-08,0.940114,1.154346,0.953727,1.065028,0.943861,0.938080,0.920829,3.267692,1.293211,0.190422,...,1.210891,0.674351,0.992177,1.026943,1.277887,0.888136,0.953666,0.974684,0.886876,0.881242
2022-09-09,0.940114,1.172423,0.961918,1.093658,0.972870,0.963532,0.935569,3.370769,1.279176,0.184721,...,1.239604,0.690420,0.992979,1.026943,1.284394,0.890395,0.965718,0.975657,0.909271,0.899635
2022-09-12,0.940114,1.203412,0.965502,1.106579,0.996507,0.981939,0.931806,3.495385,1.295216,0.189282,...,1.165347,0.706490,0.992979,1.027979,1.291569,0.889830,0.972309,0.977605,0.935452,0.910671


In [8]:
ticker_list=data_df.columns
print(f"{len(symbols)-len(ticker_list)} stocks were droped")

1642 stocks were droped


# preprocessing 


1. ticker: this is also used to select which stocks come to the model

2. price: we take the ratio: (importantly we did drop some potentialy important indicators)

3. date: the day and month are floating point numbers. while year and day of the week are embedings.



In [9]:
look=layers.StringLookup(vocabulary=ticker_list,num_oov_indices=0)
inv_look=layers.StringLookup(vocabulary=ticker_list,invert=True,num_oov_indices=0)
tick_embed=layers.Embedding(len(ticker_list)+1,512,embeddings_constraint= tf.keras.constraints.unit_norm)
day_embed=layers.Embedding(7,10)

In [10]:
dataset=tf.data.Dataset.from_tensor_slices(ticker_list)
exmple=dataset.batch(17).take(1).get_single_element()
exmple

<tf.Tensor: shape=(17,), dtype=string, numpy=
array([b'BLTSU', b'NHTC', b'PCSB', b'SLAB', b'EBIZ', b'IEV', b'OGN',
       b'PERI', b'MGYR', b'SIEB', b'IFRA', b'MCG', b'PNACU', b'XBB',
       b'AOTG', b'SNA', b'GGLS'], dtype=object)>

In [11]:
#data_df[exmple.numpy().astype('U13')]

In [12]:
def thread_distribute(l,f,n=1):
    if(n==0):
        return f(l)
    
    with concurrent.futures.ThreadPoolExecutor() as executor: 
        l=[executor.submit(lambda x: thread_distribute(x,f,n-1),x) for x in l]
        l= [x.result() for x in l]
    return l

thread_distribute([range(5) for i in range(2)],lambda x: range(x),2)

[[range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)],
 [range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)]]

In [13]:
def choose_stocks(x):
    a=tick_embed(look(ticker_list))
    b=tick_embed(look(x))

    b=b@tf.transpose(a)
    b=tf.nn.top_k(b,3)


    tickers=inv_look([list(x) for x in b[1].numpy()]).numpy().astype('U13')
    
    return [[data_df[x] for x in sub] for sub in tickers]
    #return thread_distribute(tickers,lambda x: data_df[x],2)

a=choose_stocks(exmple)

pd.DataFrame(a[0]).transpose().dropna()

,BLTSU,FRLA,INTC
2021-12-27,0.982890,1.000000,0.885675
2021-12-28,0.975285,0.997988,0.882605
2021-12-29,0.975285,1.000000,0.883799
2021-12-30,0.975285,1.000000,0.882265
2021-12-31,0.972433,1.000503,0.878172
...,...,...,...
2022-09-07,0.940114,1.020121,0.522640
2022-09-08,0.940114,1.021127,0.524345
2022-09-09,0.940114,1.025151,0.536452
2022-09-12,0.940114,1.025151,0.538157


In [14]:
a[0][0].index

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15',
               ...
               '2022-08-30', '2022-08-31', '2022-09-01', '2022-09-02',
               '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09',
               '2022-09-12', '2022-09-13'],
              dtype='datetime64[ns]', length=680, freq=None)

In [15]:
def time_encode(a):
    #a=a.index
    return (a.day_of_week,a.day/30,a.month/12,(a.year-2020))
    
time_encode(data_df.index[0])

(3, 0.06666666666666667, 0.08333333333333333, 0)

In [16]:
def time_tensors(a):
    x=thread_distribute(a,lambda x:time_encode(x.index),2)
    x=[thread_distribute(x,lambda x: x[i],2) for i in range(4)]
    return [tf.constant(x) for x in x]

time_tensors(a)

[<tf.Tensor: shape=(17, 3, 680), dtype=int32, numpy=
 array([[[3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1]],
 
        [[3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1]],
 
        [[3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1]],
 
        ...,
 
        [[3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1]],
 
        [[3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1]],
 
        [[3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1],
         [3, 4, 0, ..., 4, 0, 1]]])>,
 <tf.Tensor: shape=(17, 3, 680), dtype=float64, numpy=
 array([[[0.06666667, 0.1       , 0.2       , ..., 0.3       ,
          0.4       , 0.43333333],
         [0.06666667, 0.1       , 0.2       , ..., 0.3       ,
          0.4       , 0.43333333],
         [0.06666667, 0.1       , 0.2       , ...,

# time handeling 

problems with the data in curent time format:

1. there are null entries especialy from before a company was made 
2. the time diffrence between entries isnt clean

In [17]:
a[0][0][5:10]

2020-01-09   NaN
2020-01-10   NaN
2020-01-13   NaN
2020-01-14   NaN
2020-01-15   NaN
Name: BLTSU, dtype: float64

In [18]:
data_df[exmple.numpy().astype('U13')]

,BLTSU,NHTC,PCSB,SLAB,EBIZ,IEV,OGN,PERI,MGYR,SIEB,IFRA,MCG,PNACU,XBB,AOTG,SNA,GGLS
2020-01-02,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,1.000000,NaN
2020-01-03,NaN,1.037037,0.995547,0.980407,0.995498,0.989224,NaN,1.009231,1.000000,0.980616,0.998031,NaN,NaN,NaN,NaN,0.988698,NaN
2020-01-06,NaN,1.031482,0.992578,0.977198,0.994703,0.992816,NaN,1.123077,1.000000,1.003421,0.997116,NaN,NaN,NaN,NaN,0.981420,NaN
2020-01-07,NaN,1.038889,0.974270,0.989697,1.004767,0.987112,NaN,1.193846,1.000000,0.978335,0.994619,NaN,NaN,NaN,NaN,0.975089,NaN
2020-01-08,NaN,1.029630,0.980703,0.990710,1.008475,0.989013,NaN,1.273846,1.000000,0.969213,0.989449,NaN,NaN,NaN,NaN,0.985858,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,0.940114,1.211159,0.950144,1.003125,0.939027,0.941034,0.908283,3.290769,1.302233,0.183580,1.362513,0.429700,1.012085,0.957730,1.049986,1.421108,0.975000
2022-09-08,0.940114,1.154346,0.953727,1.065028,0.943861,0.938080,0.920829,3.267692,1.293211,0.190422,1.365102,0.469984,1.012085,0.960962,1.068199,1.413986,0.986292
2022-09-09,0.940114,1.172423,0.961918,1.093658,0.972870,0.963532,0.935569,3.370769,1.279176,0.184721,1.387293,0.473144,1.001007,0.963634,1.107540,1.420853,0.963400
2022-09-12,0.940114,1.203412,0.965502,1.106579,0.996507,0.981939,0.931806,3.495385,1.295216,0.189282,1.400977,0.486572,1.012085,0.965043,1.123690,1.429884,0.963856


In [19]:
c=np.array(["KNX","ISRA"])
c.shape

(2,)

In [20]:
na=data_df.isna().any()
na

BLTSU      True
NHTC      False
PCSB      False
SLAB      False
EBIZ      False
          ...  
NXDT      False
XDSQ       True
IPOF.U     True
OMAB      False
VIDI      False
Length: 10571, dtype: bool

In [21]:
pd.DataFrame(a[0]).transpose().dropna()

,BLTSU,FRLA,INTC
2021-12-27,0.982890,1.000000,0.885675
2021-12-28,0.975285,0.997988,0.882605
2021-12-29,0.975285,1.000000,0.883799
2021-12-30,0.975285,1.000000,0.882265
2021-12-31,0.972433,1.000503,0.878172
...,...,...,...
2022-09-07,0.940114,1.020121,0.522640
2022-09-08,0.940114,1.021127,0.524345
2022-09-09,0.940114,1.025151,0.536452
2022-09-12,0.940114,1.025151,0.538157


In [22]:
mask=np.logical_not(data_df.isna().to_numpy())
mask

array([[False,  True,  True, ..., False,  True,  True],
       [False,  True,  True, ..., False,  True,  True],
       [False,  True,  True, ..., False,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [23]:
def choose_stocks_df(x):
    a=tick_embed(look(ticker_list))
    b=tick_embed(look(x))

    b=b@tf.transpose(a)
    b=tf.nn.top_k(b,3)


    tickers=inv_look([list(x) for x in b[1].numpy()]).numpy().astype('U13')
    
    
    return [pd.DataFrame([data_df[x] for x in sub]).transpose().dropna() for sub in tickers]

v=choose_stocks_df(exmple)
v[0]

,BLTSU,FRLA,INTC
2021-12-27,0.982890,1.000000,0.885675
2021-12-28,0.975285,0.997988,0.882605
2021-12-29,0.975285,1.000000,0.883799
2021-12-30,0.975285,1.000000,0.882265
2021-12-31,0.972433,1.000503,0.878172
...,...,...,...
2022-09-07,0.940114,1.020121,0.522640
2022-09-08,0.940114,1.021127,0.524345
2022-09-09,0.940114,1.025151,0.536452
2022-09-12,0.940114,1.025151,0.538157


In [24]:
#it is clear that there isnt perfect index overlap. we need to indecies 1 by 1
#pd.concat(v).index 
t=[x.index for x in v]
t=thread_distribute(t,time_encode,2)
#t=[[x[i] for x in t] for i in range(4)]
t=[thread_distribute(t,lambda x: x[i],2) for i in range(4)]
#thread_distribute(t,len,2)
t=[tf.ragged.constant(x) for x in t]

"padding"
t=[(x+1).to_tensor()-1 for x in t]


[x.dtype for x in t]

[tf.int32, tf.float32, tf.float32, tf.int32]

In [25]:
x=thread_distribute(v,lambda x: x.to_numpy(),1)
vals=tf.concat(x,axis=0)
lengths=[len(s) for s in x]
x=tf.RaggedTensor.from_row_lengths(vals,lengths)
#[x.shape for x in x]
"padding the prices"
x=(x+1).to_tensor()-1
x=tf.transpose(x,[2,0,1])
x.shape

TensorShape([3, 17, 680])

In [26]:
def get_xs(v):
    t=[x.index for x in v]
    t=thread_distribute(t,time_encode,2)
    #t=[[x[i] for x in t] for i in range(4)]
    t=[thread_distribute(t,lambda x: x[i],2) for i in range(4)]
    #thread_distribute(t,len,2)
    t=[tf.ragged.constant(x) for x in t]
    
    x=thread_distribute(v,lambda x: x.to_numpy(),1)
    vals=tf.concat(x,axis=0)
    lengths=[len(s) for s in x]
    x=tf.RaggedTensor.from_row_lengths(vals,lengths)
    #[x.shape for x in x]
    
    "padding"
    x=(x+1).to_tensor()-1
    t=[(x+1).to_tensor()-1 for x in t] 
    
    x=tf.transpose(x,[2,0,1])
    t.extend(x) 
    return t

xs=get_xs(v)
[x.shape for x in xs]

[TensorShape([17, 680]),
 TensorShape([17, 680]),
 TensorShape([17, 680]),
 TensorShape([17, 680]),
 TensorShape([17, 680]),
 TensorShape([17, 680]),
 TensorShape([17, 680])]

In [27]:
base=[x.iloc[:,0] for x in v]
y=pd.DataFrame([s[1:] for s in base])
x=pd.DataFrame([s[:-1] for s in base])

In [28]:
x.to_numpy().shape,y.to_numpy().shape

((17, 679), (17, 679))

In [29]:
[len(x) for x in v]

[180, 24, 150, 212, 359, 351, 336, 352, 680, 680, 680, 294, 83, 75, 53, 278, 6]

In [30]:
def sample_seq(x,LEN=100):
    l=len(x)
    if(l<LEN):
        return x 
    
    start=np.random.randint(l-LEN)
    return x.iloc[start:start+LEN]
sample_seq(v[-1]) 
sample_seq(v[0])

,BLTSU,FRLA,INTC
2022-03-17,0.961977,1.008048,0.803826
2022-03-18,0.961977,1.007545,0.809112
2022-03-21,0.961977,1.009054,0.808089
2022-03-22,0.961977,1.009054,0.825141
2022-03-23,0.961977,1.008048,0.823094
...,...,...,...
2022-08-03,0.943916,1.016097,0.622735
2022-08-04,0.943916,1.016097,0.608070
2022-08-05,0.943916,1.016097,0.603466
2022-08-08,0.943916,1.016097,0.603296


In [31]:
base=[sample_seq(x) for x in v]
y=[s.iloc[1:,0] for s in base]
x=[s[:-1] for s in base]
x=get_xs(x)

In [85]:
def get_numbers(tickers):
    v=choose_stocks_df(tickers)
    base=[sample_seq(x) for x in v]
    
    y=[s.iloc[1:,0].to_numpy() for s in base]
    y=tf.ragged.constant(y)
    y=(y+1).to_tensor()-1
    
    x=[s[:-1] for s in base]
    x=get_xs(x)
    x.append(tickers)
    x.append(y)
    return x
t=time.time()
n=get_numbers(exmple)
print(time.time()-t)

0.4796903133392334


In [95]:
specs=[tf.TensorSpec.from_tensor(x) for x in n]
s=[list(x.shape) for x in specs]
for x in s:
    x[0]=None
specs=[tf.TensorSpec(s[i],specs[i].dtype) for i in range(len(specs))]
specs

[TensorSpec(shape=(None, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None)]

In [104]:
def gather_x(*args):
    return args[:-1]

def gather_y(*args):
    return args[-1]

gather_x(1,2,3,4),gather_y(1,2,3,4)

((1, 2, 3), 4)

In [120]:
dataset2=dataset.shuffle(1000).batch(32).map(lambda y: tf.py_function(get_numbers,inp=[y],Tout=specs))

#x=dataset2.take(1).get_single_element()
dataset_x=dataset2.map(gather_x)
dataset_y=dataset2.map(gather_y)
dataset2=tf.data.Dataset.zip((dataset_x,dataset_y))
s
x=dataset2.take(1).get_single_element()